# 8 /// Intervalles d'Allen ////////////////////////////////////////////////////////////////////////////////////////////////                                       

### _/// Exercice 1 : utilitaires généraux_

In [21]:
                                                                                 # question 1 : dictionnaire
# -----------------------------------------------------------------------------------------------------------------------------
# Charger un dictionnaire des relations valables, de leurs transposées et de leurs symétriques.

# transpose : dict[str:str]
transpose = {   '<':'>',
                '>':'<',
                'e':'et',
                's':'st',
                'et':'e',
                'st':'s',
                'd':'dt',
                'm':'mt',
                'dt':'d',
                'mt':'m',
                'o':'ot',
                'ot':'o',
                '=':'='     }

# symetrie : dict[str:str]
symetrie = {    '<':'>',
                '>':'<',
                'e':'s',
                's':'e',
                'et':'st',
                'st':'et',
                'd':'d',
                'm':'mt',
                'dt':'dt',
                'mt':'m',
                'o':'ot',
                'ot':'o',
                '=':'='     }            

# compositionBase : dict[tuple[str,str]:set[str]]             
compositionBase = {     ('<','<'):{'<'},
                        ('<','m'):{'<'},
                        ('<','o'):{'<'},
                        ('<','et'):{'<'},
                        ('<','s'):{'<'},
                        ('<','d'):{'<','m','o','s','d'},
                        ('<','dt'):{'<'},
                        ('<','e'):{'<','m','o','s','d'},
                        ('<','st'):{'<'},
                        ('<','ot'):{'<','m','o','s','d'},
                        ('<','mt'):{'<','m','o','s','d'},
                        ('<','>'):{'<','>','m','mt','o','ot','e','et','s','st','d','dt','='},
                        ('m','m'):{'<'},
                        ('m','o'):{'<'},
                        ('m','et'):{'<'},
                        ('m','s'):{'m'},
                        ('m','d'):{'o','s','d'},
                        ('m','dt'):{'<'},
                        ('m','e'):{'o','s','d'},
                        ('m','st'):{'m'},
                        ('m','ot'):{'o','s','d'},
                        ('m','mt'):{'e','et','='},
                        ('o','o'):{'<','m','o'},
                        ('o','et'):{'<','m','o'},
                        ('o','s'):{'o'},
                        ('o','d'):{'o','s','d'},
                        ('o','dt'):{'<','m','o','et','dt'},
                        ('o','e'):{'o','s','d'},
                        ('o','st'):{'o','et','dt'},
                        ('o','ot'):{'o','ot','e','et','d','dt','st','s','='},
                        ('s','et'):{'<','m','o'},
                        ('s','s'):{'s'},
                        ('s','d'):{'d'},
                        ('s','dt'):{'<','m','o','et','dt'},
                        ('s','e'):{'d'},
                        ('s','st'):{'s','st','='},
                        ('et','s'):{'o'},
                        ('et','d'):{'o','s','d'},
                        ('et','dt'):{'dt'},
                        ('et','e'):{'e','et','='},
                        ('d','d'):{'d'},
                        ('d','dt'):{'<','>','m','mt','o','ot','e','et','s','st','d','dt','='},
                        ('dt','d'):{'o','ot','e','et','d','dt','st','s','='}       }

# -----------------------------------------------------------------------------------------------------------------------------

In [22]:
                                                                                    # question 2 : transposées, symétriques
# -----------------------------------------------------------------------------------------------------------------------------
def transposeSet(S) :
    R=set()
    for i in S:
        R.add(transpose[i])
    return R
def symetrieSet(S) :
    R=set()
    for i in S:
        R.add(symetrie[i])
    return R
# -----------------------------------------------------------------------------------------------------------------------------

In [23]:
print(transposeSet({'s','d','<','et'}))  
print(symetrieSet({'s','d','<','et'}))
print(transposeSet(transposeSet('s')|transposeSet('e')))

{'>', 'e', 'st', 'dt'}
{'>', 'e', 'st', 'd'}
{'e', 's'}


In [24]:
                                                                                   # question 3 : notion de composition
# -----------------------------------------------------------------------------------------------------------------------------
def compose(r1,r2):
    t1=transpose[r1]
    t2=transpose[r2]
    s1=symetrie[r1]
    s2=symetrie[r2]
    st1=transpose[s1]
    st2=transpose[s2]
    CB = compositionBase.keys()
    
    if r1 == '=':
        return {r2}
    elif r2 == '=':
        return {r1}
    elif (r1,r2) in CB:
        return compositionBase[(r1,r2)]   
    elif (t2,t1) in CB:
        return transposeSet(compositionBase[(t2,t1)])
    elif (s1,s2) in CB:
        return symetrieSet(compositionBase[(s1,s2)])
    elif (st2,st1) in CB:
        return symetrieSet(transposeSet(compositionBase[(st2,st1)]))
# -----------------------------------------------------------------------------------------------------------------------------

                                                                                   # question 4 : généralisation
# -----------------------------------------------------------------------------------------------------------------------------
def compositionSet(S1,S2):
    R = set()
    for r1 in S1:
        for r2 in S2:
            R |= compose(r1,r2)
    return R
# -----------------------------------------------------------------------------------------------------------------------------

In [25]:
print(compose('=','d'))
print(compose('m','d'))
print(compose('ot','>'))
print(compose('>','e'))
print(compose('ot','m'))
print(compositionSet({'=','o'},{'dt','st','et','='}))
print(compositionSet({'=','e','et'},{'=','e','d','s'}))
# exemple vu en travaux dirigés
print(compositionSet({'m','o'},{'dt','st','et','='}))

{'d'}
{'o', 's', 'd'}
{'>'}
{'>'}
{'o', 'et', 'dt'}
{'o', '<', 'm', '=', 'st', 'dt', 'et'}
{'o', 's', 'e', '=', 'et', 'd'}
{'o', '<', 'm', 'dt', 'et'}


### _/// Exercice 2 : génération et modification de graphes_

In [26]:
class Graphe(object):
    # définition d'un objet avec un ensemble de noeuds et de relations
    def __init__(self,noeuds=set(),relations=dict()):
        self.noeuds = noeuds
        self.relations = relations
                                                                                   # question 1 : récupération des relations
# -----------------------------------------------------------------------------------------------------------------------------
    def getRelations(self,i,j):
        Rij = self.relations.get((i,j),{})
        if len(Rij) == 0: # pas de relation entre i et j...
            # ...a-t-on la transposée ?
            Rij = transposeSet(self.relations.get((j,i),{}))
            if len(Rij) != 0:
                # soit il y a quelque chose de défini,
                return Rij
            elif i in self.noeuds and j in self.noeuds:
                # soit tout est possible
                return set(transpose.keys())
        # par défaut, retourner directement la relation existante entre i et j
        return Rij
# -----------------------------------------------------------------------------------------------------------------------------
    
                                                                                   # question 2 : propagation d'une relation
# -----------------------------------------------------------------------------------------------------------------------------
    def modification(self,i,j,newR):
        # laquelle d'(i,j) ou de (j,i) est stockée dans le graphe ? c'est celle qu'il faut mettre à jour.
        if (i,j) in self.relations.keys(): 
            self.relations[(i,j)] = newR
        else:
            self.relations[(j,i)] = transposeSet(newR)
        
def propagation(g,i,j,Rij):
    # Simplement une application de l'algorithme donné.
    g.relations[(i,j)] = Rij # étude de la relation entre i et j
    pile = [(i,j)]
    while len(pile) > 0:
        i,j = pile[0]
        del(pile[0])
        noeuds = g.noeuds.copy()
        noeuds.remove(i)
        noeuds.remove(j)
        # peut-on faire passer un chemin par k, un noeud différent de i ou j ?
        for k in noeuds: 
            Rik = g.getRelations(i,k)
            Rij = g.getRelations(i,j)
            Rjk = g.getRelations(j,k)
            Rkj = g.getRelations(k,j)
            Rki = g.getRelations(k,i)
            newRik = Rik.intersection(compositionSet(Rij,Rjk))
            newRkj = Rkj.intersection(compositionSet(Rki,Rij))
            if len(newRik) == 0 or len(newRkj) == 0:
                return  # contradiction temporelle
            if newRik != Rik:
                g.modification(i,k,newRik) # mettre à jour (i,k)
                pile.append((i,k))
            if newRkj != Rkj:
                g.modification(k,j,newRkj) # mettre à jour (k,j)
                pile.append((k,j))
# -----------------------------------------------------------------------------------------------------------------------------

                                                                                   # question 3 : ajout d'une relation
# -----------------------------------------------------------------------------------------------------------------------------
def ajouter(g,i,j,r):
    # 
    g.noeuds |= {i,j}
    Rij = g.relations.get((i,j),{})
    if len(Rij) == 0: 
        g.relations[(i,j)] = {r}
    else:
        g.relations[(i,j)] |= {r}
# -----------------------------------------------------------------------------------------------------------------------------

In [27]:
                                                                                   # question 4 : des graphes à trois noeuds
# -----------------------------------------------------------------------------------------------------------------------------
g1 = Graphe({'A','B','C'},{('A','B'):{'<'},
                           ('A','C'):{'>'}})
propagation(g1,'B','C','=')
print(g1.noeuds,g1.relations)

g2 = Graphe({'A','B','C'},{('A','B'):{'<'},
                           ('A','C'):{'<'}})
propagation(g2,'B','C','=')
print(g2.noeuds,g2.relations)
# -----------------------------------------------------------------------------------------------------------------------------   

{'B', 'A', 'C'} {('A', 'B'): {'<'}, ('A', 'C'): {'>'}, ('B', 'C'): '='}
{'B', 'A', 'C'} {('A', 'B'): {'<'}, ('A', 'C'): {'<'}, ('B', 'C'): '='}


In [36]:
                                                                                   # question 5 : une histoire de lampe
# -----------------------------------------------------------------------------------------------------------------------------
g = Graphe({'S','L','R'},{('L','S'):{'ot','mt'}, # rapport entre l'instant où j'allume et la lampe allumée
                          ('S','R'):{'<','m','mt','>'}}) # rapport entre l'instant où j'allume et la présence de John
propagation(g,'L','S',{'ot','mt'})
propagation(g,'S','R',{'<','m','mt','>'})
# On observe automatiquement la création d'une nouvelle arête : elle relie John à la lampe allumée.
print(g.noeuds,g.relations)
#...avec un tas de possibilités.
# -----------------------------------------------------------------------------------------------------------------------------

{'S', 'R', 'L'} {('L', 'S'): {'ot', 'mt'}, ('S', 'R'): {'<', '>', 'm', 'mt'}, ('R', 'L'): {'<', '>', 's', 'st', '=', 'mt', 'e', 'ot', 'd'}}


In [39]:
                                                                                   # question 6 : débarquement de John
# -----------------------------------------------------------------------------------------------------------------------------
# reprise des anciennes informations
g = Graphe({'S','L','R'},{('L','S'):{'ot','mt'},
                          ('S','R'):{'<','m','mt','>'}})
propagation(g,'L','R',{'o','s','=','m','d'}) # nouveauté : John était dans la pièce quand la lumière a été éteinte.
print(g.noeuds,g.relations)
# un peu moins de possibilités.
# -----------------------------------------------------------------------------------------------------------------------------

{'S', 'R', 'L'} {('L', 'S'): {'ot', 'mt'}, ('S', 'R'): {'<', 'm'}, ('L', 'R'): {'o', 's', 'm', '='}}


In [50]:
                                                                                   # question 7 : affaire de petit déjeuner
# -----------------------------------------------------------------------------------------------------------------------------
# Alfred mange, boit son café, lit son journal et s'en va se promener, dans un ordre fixé :
graphe = Graphe({'J','D','C','P'},{('J','D'):{'d','dt','o','ot','e','et','s','st','='}, # rapport journal - petit-déjeuner
                                   ('J','C'):{'=','e','et'}, # rapport journal-café
                                   ('D','P'):{'m','<'}}) # rapport promenade-petit-déjeuner
propagation(graphe,'C','D',{'=','s','e','d'}) # le café est une étape du petit déjeuner
print(graphe.noeuds,graphe.relations)
# Les résultats concordent.
# -----------------------------------------------------------------------------------------------------------------------------

{'P', 'J', 'D', 'C'} {('J', 'D'): {'o', 's', '=', 'et', 'e', 'd'}, ('J', 'C'): {'et', 'e', '='}, ('D', 'P'): {'<', 'm'}, ('C', 'D'): {'s', 'e', '=', 'd'}, ('P', 'C'): {'>', 'mt'}, ('P', 'J'): {'>', 'mt'}}


### _/// Conclusions_

Ce huitième TME traite d'un sujet distinct des précédents : la représentation du temps, grâce aux intervalles d'Allen. C'est une façon de dénoter des relations (treize possibilités) entre des événements, qui se traduit sous la forme d'un graphe.   
Le graphe montre toutes les relations, parmi les treize, que l'on peut envisager entre deux points nommés du temps (l'instant où j'écris ces lignes et celui où j'ai pris mon déjeuner) : on les lit alors sur l'arête qui les relie. J'ai pu écrire après manger (mauvaise idée), en mangeant (d'où les taches sur mon clavier) ou avant (sage décision), avec toutes les nuances qu'impliquent ces relations (les intervalles peuvent finir en même temps, l'un un tout petit peu avant l'autre, et caetera).  

Connaissant le rapport entre une paire d'événements donnés, il est possible de déduire d'autres propriétés à propos de tous ceux qui leurs sont liés. En particulier, toute nouvelle information peut se traduire par une contrainte supplémentaire que l'on *propage* à travers le graphe. La propagation d'une information revient en fait à restreindre les possibilités associée à chaque arête et donc à s'assurer plus de certitudes.  

Ce raisonnement nous aide à résoudre des problèmes divers, par exemple, concrètement, la génération d'un emploi du temps qui réponde à certaines demandes : tel cours doit précéder tel TD, qui ne peut pas être en conflit avec tel autre, pour des raisons d'accessibilité aux étudiants... Ici, on se penche sur des exemples-jouets, moins pragmatiques, celui d'une lampe ou d'un petit déjeuner. Avec une liste d'événements et des informations incomplètes, les intervalles d'Allen permettent alors de mener l'enquête sur ce qui s'est passé, et de raisonner sur les ordres possibles.  